# Chapter 4 - Decision Trees

Paul E. Anderson

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

## How would you communicate your decision about a typical Thursday night?

Let's pretend you have a new roommate who is part of an exchange program from your favorite foreign city. They are a little confused about what to do on their first Thursday night in town, so they ask you for help. You talk to them for a while, but it becomes clear they really won't leave you alone until you diagram your decision making progress. Let's take 5-10 minutes and come up with a tree you can share with the class :)

Here are your features:
* Deadline? $\in$ {Urgent, Near, None}
* Lazy? $\in$ {Yes, No}
* Party? $\in$ {Yes, No party going on tonight}

You are trying to predict one of the following activities:
* Activity $\in$ {Party, TV, Pub, Study}

### What if we needed a way to create a tree that generalized to many people?

Onto our decision tree algorithm!

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(f'{home}/csc-466-student/data/activity.csv')
df

,Deadline?,Is there a party?,Lazy?,Activity
0,Urgent,Yes,Yes,Party
1,Urgent,No,Yes,Study
2,Near,Yes,Yes,Party
3,None,Yes,No,Party
4,None,No,Yes,Pub
5,None,Yes,No,Party
6,Near,No,No,Study
7,Near,No,Yes,TV
8,Near,Yes,Yes,Party
9,Urgent,No,No,Study


### Let's play a game of 20 questions

You've got 20 questions and you are trying to quess what your opponent is thinking by asking questions. 

For each question, you are trying to ask something that gives you the most information given what you already know. 

It is common to ask things like "Is it an animal?" before asking "Is it a cat?"

We need a way to encode this mathematically, and this leads us to information theory.

We will reserve the writing of the decision tree algorithm to lab this week. Instead, we will focus on defining the process mathematically. To do this, we need to start discussing sets and metrics we can use about sets.

### Entropy

We will first discuss the entropy of the universe. If said universe consisted of a set of items. 

Let $S$ be a set of items. Each element of $S$ can take on one of $Y$ values. We will write the unique values of $Y$ as $V_Y$.

$p(y)$ proportion of the number of elements in class ${\displaystyle y}$ to the number of elements in set

${\displaystyle \mathrm {H} {(S)}=\sum _{y\in V_Y}{-p(y)\log _{2}p(y)}}$

Let's look at our concrete example:

In [4]:
S = df['Activity'].values
S

array(['Party', 'Study', 'Party', 'Party', 'Pub', 'Party', 'Study', 'TV',
       'Party', 'Study'], dtype=object)

In [5]:
VY = df['Activity'].unique()
VY

array(['Party', 'Study', 'Pub', 'TV'], dtype=object)

**Stop and think:** Can you write a loop to calculate entropy using ``VY`` and ``S``?

In [6]:
entropy = 0
# Your solution here
entropy

1.6854752972273346

**Stop and think:** In your own words, how would you calculate the entropy after asking whether there is a deadline?

**Your answer here**

**Note:** When calculating entropy for classification in a decision tree, you will always calculate the entropy on the column you are trying to predict.

In [7]:
df['Deadline?'].unique()

array(['Urgent', 'Near', 'None'], dtype=object)

**Let's now do the entropy calculation for only those samples with an urgent deadline.**

In [8]:
df_urgent = df.loc[df['Deadline?'] == 'Urgent']
df_urgent

,Deadline?,Is there a party?,Lazy?,Activity
0,Urgent,Yes,Yes,Party
1,Urgent,No,Yes,Study
9,Urgent,No,No,Study


In [9]:
def calc_entropy(S):
    VY = S.unique()
    entropy = 0
    # Your solution here
    return entropy

In [10]:
calc_entropy(df['Activity'])

1.6854752972273346

In [11]:
calc_entropy(df_urgent['Activity'])

0.9182958340544896

**Stop and think:** What is the exact question we are trying to answer?

**Your solution here**

Now we will print out the conditional entropy for each unique value of deadline. This is called the *specific conditional entropy*.

In [12]:
print(calc_entropy(df['Activity']))
cond_entropy = 0
for value in df['Deadline?'].unique():
    df2 = df.loc[df['Deadline?'] == value]
    print(value,calc_entropy(df2['Activity']),len(df2),len(df2)/len(df)*calc_entropy(df2['Activity']))
    cond_entropy += len(df2)/len(df)*calc_entropy(df2['Activity'])
cond_entropy

1.6854752972273346
Urgent 0.9182958340544896 3 0.27548875021634683
Near 1.5 4 0.6000000000000001
None 0.9182958340544896 3 0.27548875021634683


1.1509775004326936

**We can put this into a function:**

In [15]:
def calc_cond_entropy(X,S):
    cond_entropy = 0
    for value in X.unique():
        S_cond_value = S[X == value]
        cond_entropy += len(S_cond_value)/len(S)*calc_entropy(S_cond_value)
    return cond_entropy

In [16]:
calc_cond_entropy(df['Deadline?'],df['Activity'])

1.1509775004326936

In [17]:
calc_cond_entropy(df['Is there a party?'],df['Activity'])

0.6854752972273344

In [18]:
calc_cond_entropy(df['Lazy?'],df['Activity'])

1.4754887502163467

**Stop and think:** How do you calculate the entropy knowing the specific value for a question such as deadline?

Your solution here

**Stop and think:** How do you measure the entropy after you ask a question such as deadline?

Your solution here

### Now onto the ID3 algorithm!
From Marsland Chapter 12:


* If all examples have the same label:
    * return a leaf with that label
* Else if there are no features left to test:
    * return a leaf with the most common label
* Else:
    * choose the feature named $F$ (i.e., column) that maximises the information gain of $S$ to be the next node.
    * add a branch from the current node for each unique value f in $X_F$
    * for each branch:
        * calculate $S_f$ by removing $F$ from the set of features
        * recursively call the algorithm with $S_f$ , to compute the gain relative to the current set of examples

Let's now define information gain as the difference between the entropy prior to the split to the entropy after the split.

In [19]:
def information_gain(X,S):
    orig_entropy = calc_entropy(S)
    cond_entropy = calc_cond_entropy(X,S)
    return orig_entropy - cond_entropy

In [20]:
information_gain(df['Lazy?'],df['Activity'])

0.20998654701098785

In [21]:
information_gain(df['Is there a party?'],df['Activity'])

1.0000000000000002

In [22]:
information_gain(df['Deadline?'],df['Activity'])

0.5344977967946409

### Let's make our tree!

To the whiteboard or computer! Below is some scratch work to get us started. Once you have the pattern, you can continue on your own to practice. We will store our tree in a dictionary.

In [37]:
tree = {"Is there a party?":{"Yes":[],"No":[]}}
import json
print(json.dumps(tree,sort_keys=True, indent=4))

{
    "Is there a party?": {
        "No": [],
        "Yes": []
    }
}


In [39]:
new_df = df.loc[df['Is there a party?'] == 'No'].drop('Is there a party?',axis=1)
new_df

,Deadline?,Lazy?,Activity
1,Urgent,Yes,Study
4,None,Yes,Pub
6,Near,No,Study
7,Near,Yes,TV
9,Urgent,No,Study


In [40]:
information_gain(new_df['Lazy?'],new_df['Activity'])

0.41997309402197514

In [41]:
information_gain(new_df['Deadline?'],new_df['Activity'])

0.9709505944546687

In [43]:
new_tree = {"Deadline?":{"Urgent":None,"Near":None,"None":None}}
print(json.dumps(new_tree,sort_keys=True, indent=4))

{
    "Deadline?": {
        "Near": null,
        "None": null,
        "Urgent": null
    }
}


In [45]:
tree['Is there a party?']['No'] = new_tree
print(json.dumps(tree,sort_keys=True, indent=4))

{
    "Is there a party?": {
        "No": {
            "Deadline?": {
                "Near": null,
                "None": null,
                "Urgent": null
            }
        },
        "Yes": []
    }
}


In [47]:
new_df2 = new_df.loc[new_df['Deadline?'] == 'None'].drop('Deadline?',axis=1)
new_df2

,Lazy?,Activity
4,Yes,Pub


Does this hit a base case?

Answer: Yes. There is only one label and it is Pub.

In [48]:
tree['Is there a party?']['No']['Deadline?']['None'] = "Pub"
print(json.dumps(tree,sort_keys=True, indent=4))

{
    "Is there a party?": {
        "No": {
            "Deadline?": {
                "Near": null,
                "None": "Pub",
                "Urgent": null
            }
        },
        "Yes": []
    }
}


In [49]:
new_df2 = new_df.loc[new_df['Deadline?'] == 'Near'].drop('Deadline?',axis=1)
new_df2

,Lazy?,Activity
6,No,Study
7,Yes,TV


In [50]:
information_gain(new_df2['Lazy?'],new_df2['Activity'])

1.0

In [51]:
new_df3 = new_df2.loc[new_df2['Lazy?'] == 'Yes'].drop('Lazy?',axis=1)
new_df3

,Activity
7,TV


In [53]:
tree['Is there a party?']['No']['Deadline?']['Near'] = {"Lazy?":{"No":None,"Yes":None}}
print(json.dumps(tree,sort_keys=True, indent=4))

{
    "Is there a party?": {
        "No": {
            "Deadline?": {
                "Near": {
                    "Lazy?": {
                        "No": null,
                        "Yes": null
                    }
                },
                "None": "Pub",
                "Urgent": null
            }
        },
        "Yes": []
    }
}


In [54]:
tree['Is there a party?']['No']['Deadline?']['Near']['Lazy?']['Yes'] = 'TV'
print(json.dumps(tree,sort_keys=True, indent=4))

{
    "Is there a party?": {
        "No": {
            "Deadline?": {
                "Near": {
                    "Lazy?": {
                        "No": null,
                        "Yes": "TV"
                    }
                },
                "None": "Pub",
                "Urgent": null
            }
        },
        "Yes": []
    }
}


### Advances

Now that we have a basic algorithm, what are some of the advances:
* Continuous variables
* C4.5 with pruning

**Stop and think:** How would you adapt for continuous variables?

So our next natural question is ... how do we compare algorithms?

##### Your solution here

## Practice Problems

**Practice problem 1:** Construct the full decision tree and show all of the calculations of entropy and information gain for the following data set. Show all of your work. 

In [5]:
import pandas as pd
table = pd.DataFrame([
    [1,"Not included",3,"New","Yes"],
    [2,"Included",3,"Old","No"],
    [3,"Not included",4,"Old","Yes"],
    [4,"Not included",3,"Old","No"],
    [5,"Included",4,"Old","Yes"],    
],columns=["ID","Furniture/F","# Rooms/R","Kitchen/K","Acceptable"]).set_index("ID")
table

,Furniture/F,# Rooms/R,Kitchen/K,Acceptable
ID,,,,
1,Not included,3,New,Yes
2,Included,3,Old,No
3,Not included,4,Old,Yes
4,Not included,3,Old,No
5,Included,4,Old,Yes
